In [ ]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc 
import dash_html_components as html

# from pandas_datareader import data as web 
import yfinance as yf
from datetime import datetime as dt
print('hello')
app = dash.Dash('Hello World',
                external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

app.layout = html.Div([
    dcc.Dropdown(
        id='my-dropdown',
        options=[
            {'label': 'Coke', 'value': 'COKE'},
            {'label': 'Tesla', 'value': 'TSLA'},
            {'label': 'Apple', 'value': 'AAPL'}
        ],  
        value='TSLA'
    ),  
    dcc.Graph(id='my-graph')
], style={'width': '500'})

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown_value):
#     df = web.DataReader(
#         selected_dropdown_value,
#         'yahoo',
#         dt(2017, 1, 1), 
#         dt.now()
#     )   
    df =  yf.download(selected_dropdown_value, start='2020-01-01', proxy="127.0.0.1:33210")
    return {
        'data': [{
            'x': df.index,
            'y': df.Close
        }], 
        'layout': {'margin': {'l': 40, 'r': 0, 't': 20, 'b': 30}}
    }   

if __name__ == '__main__':
    app.run_server()

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import yfinance as yf

app = dash.Dash()
# df =  yf.download('AAPL', start='2020-01-01', proxy="127.0.0.1:33210")
df = pd.read_csv(
    'https://gist.githubusercontent.com/chriddyp/'
    'cb5392c35661370d95f300086accea51/raw/'
    '8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/'
    'indicators.csv')

available_indicators = df['Indicator Name'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        step=None,
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])

@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    return {
        'data': [go.Scatter(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

def create_time_series(dff, axis_type, title):
    return {
        'data': [go.Scatter(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)

@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
D:\anaconda\envs\dissertation\lib\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources

127.0.0.1 - - [19/Jul/2022 15:23:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2022 15:23:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2022 15:23:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2022 15:23:49] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Jul/2022 15:23:49] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Jul/2022 15:23:49] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0

In [10]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc 
import dash_html_components as html

# from pandas_datareader import data as web 
import yfinance as yf
from datetime import datetime as dt

import numpy as np 
print('hello')
app = dash.Dash(__name__,
                external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
dcc_slider_year = np.linspace(2000,2021,1)
app.layout = html.Div([
    dcc.Dropdown(
        id='my-dropdown',
        options=[
            {'label': 'Coke', 'value': 'COKE'},
            {'label': 'Tesla', 'value': 'TSLA'},
            {'label': 'Apple', 'value': 'AAPL'}
        ],  
        value='TSLA'
    ),  
    dcc.Graph(id='my-graph'),
    dcc.Slider(
        id='Start-Year',
        min=2000,
        max=2022,
        value=2021,
#         marks = {2000,2001,2002,2003,2004,2005,2021},
#         marks = {{str(year): str(year) for year in df['year'].unique()},}
        step=1,)
], style={'width': '500'})

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown_value):
#     df = web.DataReader(
#         selected_dropdown_value,
#         'yahoo',
#         dt(2017, 1, 1), 
#         dt.now()
#     )   
#     return {
#         'data': [{
#             'x': df.index,
#             'y': df.Close
#         }], 
#         'layout': {'margin': {'l': 40, 'r': 0, 't': 20, 'b': 30}}
#     }   
    df =  yf.download(selected_dropdown_value, start='2020-01-01', proxy="127.0.0.1:33210")

    import pandas as pd
# df = yf.download(symbol, start='2020-01-01', proxy="127.0.0.1:33210")
    print(df)
    from ta.trend import MACD
    from ta.momentum import StochasticOscillator
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
    fig.add_trace(go.Candlestick(x=df.index,
      open=df['Open'],
      high=df['High'],
      low=df['Low'],
      close=df['Close'],
      showlegend=False), row=1, col=1)

    #把非交易日排除掉
    # hide weekends
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])]) 
    # removing all empty dates
    # build complete timeline from start date to end date
    dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
    # retrieve the dates that ARE in the original datset
    dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
    # define dates with missing values
    dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
    fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

    #add moving average
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA5'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=2), 
                         name='MA 5'), row=1, col=1)

    fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA20'], 
                         opacity=0.7, 
                         line=dict(color='orange', width=2), 
                         name='MA 20'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA50'], 
                         opacity=0.7, 
                         line=dict(color='pink', width=2), 
                         name='MA 50'), row=1, col=1)


    # MACD
    macd = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)
    # stochastic
    stoch = StochasticOscillator(high=df['High'],
                             close=df['Close'],
                             low=df['Low'],
                             window=14, 
                             smooth_window=3)


    # Plot OHLC on 1st subplot (using the codes from before)
    # Plot volume trace on 2nd row 
    fig.add_trace(go.Bar(x=df.index, 
                     y=df['Volume']
                    ), row=2, col=1)
    # Plot MACD trace on 3rd row
    fig.add_trace(go.Bar(x=df.index, 
                     y=macd.macd_diff()
                    ), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd(),
                         line=dict(color='black', width=2)
                        ), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd_signal(),
                         line=dict(color='blue', width=1)
                        ), row=3, col=1)
    # Plot stochastics trace on 4th row
    fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch(),
                         line=dict(color='black', width=2)
                        ), row=4, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch_signal(),
                         line=dict(color='blue', width=1)
                        ), row=4, col=1)
    #传统的stochastics oscillator 80超买，20超卖,把这个加进stochastics
    
    const1 = np.linspace(start=20, stop=20,num=len(df.index))
    const2 = np.linspace(start=80, stop=80,num=len(df.index))
    fig.add_trace(go.Scatter(x=df.index,
                         y=const1,
                         line=dict(color='red',width=1)
                        ),row=4,col=1
                 )
    fig.add_trace(go.Scatter(x=df.index,
                         y=const2,
                         line=dict(color='green',width=1)
                        ),row=4,col=1
                 )
    # update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
    fig.update_layout(height=900, width=1200, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(values=dt_breaks)])
    return fig

if __name__ == '__main__':
    app.run_server()

hello
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [19/Jul/2022 16:48:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2022 16:48:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2022 16:48:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2022 16:48:51] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Jul/2022 16:48:51] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Jul/2022 16:48:51] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Jul/2022 16:48:51] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-12-31   81.000000   84.258003   80.416000   83.666000   83.666000   
2020-01-02   84.900002   86.139999   84.342003   86.052002   86.052002   
2020-01-03   88.099998   90.800003   87.384003   88.601997   88.601997   
2020-01-06   88.094002   90.311996   88.000000   90.307999   90.307999   
2020-01-07   92.279999   94.325996   90.671997   93.811996   93.811996   
...                ...         ...         ...         ...         ...   
2022-07-12  710.539978  719.320007  685.109985  699.210022  699.210022   
2022-07-13  676.500000  726.179993  675.099976  711.119995  711.119995   
2022-07-14  704.690002  715.960022  688.000000  714.940002  714.940002   
2022-07-15  720.000000  730.869995  710.669983  720.200012  720.200012   
2022-07-18  734.809998  751.549988  718.809

127.0.0.1 - - [19/Jul/2022 16:48:52] "POST /_dash-update-component HTTP/1.1" 200 -


[*********************100%***********************]  1 of 1 completed

127.0.0.1 - - [19/Jul/2022 16:48:59] "POST /_dash-update-component HTTP/1.1" 200 -



                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2019-12-31  288.579987  290.750000  283.130005  284.049988  281.764191   41900
2020-01-02  285.739990  286.290009  277.329987  282.399994  280.127472   42300
2020-01-03  279.769989  287.359985  277.480011  285.760010  283.460449   37500
2020-01-06  284.049988  286.500000  281.640015  282.010010  279.740662   29300
2020-01-07  281.320007  282.559998  275.380005  279.160004  276.913544   42400
...                ...         ...         ...         ...         ...     ...
2022-07-12  567.219971  597.489990  558.039978  569.359985  569.359985   44500
2022-07-13  564.989990  590.750000  558.010010  579.280029  579.280029   27600
2022-07-14  537.989990  544.000000  483.779999  514.320007  514.320007  216800
2022-07-15  525.130005  526.770020  487.859985  501.000000  501.000000  138600
2022-07-18  513.000000  514.979980  490.000000  495

127.0.0.1 - - [19/Jul/2022 16:49:04] "POST /_dash-update-component HTTP/1.1" 200 -



                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-12-31   72.482498   73.419998   72.379997   73.412498   72.139946   
2020-01-02   74.059998   75.150002   73.797501   75.087502   73.785919   
2020-01-03   74.287498   75.144997   74.125000   74.357498   73.068558   
2020-01-06   73.447502   74.989998   73.187500   74.949997   73.650795   
2020-01-07   74.959999   75.224998   74.370003   74.597504   73.304405   
...                ...         ...         ...         ...         ...   
2022-07-12  145.759995  148.449997  145.050003  145.860001  145.860001   
2022-07-13  142.990005  146.449997  142.119995  145.490005  145.490005   
2022-07-14  144.080002  148.949997  143.250000  148.470001  148.470001   
2022-07-15  149.779999  150.860001  148.199997  150.169998  150.169998   
2022-07-18  150.740005  151.570007  146.699997  147.070007  147.070007   

               Volume  
Date        